<a href="https://colab.research.google.com/github/aneeshoberoi/churnhw/blob/main/ChurnNoteBook_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.float_format', lambda x: '%.0f' % x)
pd.set_option("display.precision",2)

In [3]:
df=pd.read_csv('https://raw.githubusercontent.com/aneeshoberoi/churnhw/main/churn_homework%20-%20homework_orders.csv?token=AP7KM2WDNY5XXC7AIA4M2327SSO42', sep=",")

In [5]:
test=df.head(100)
test.set_index('order_id')

,source,date_purchased,shipping_type,payment_method,promo_value,restaurant_total,customer,shops_id,state,postal_code
order_id,,,,,,,,,,
47433960,iosapp,2019-03-13T08:02:07.000Z,Delivery,credit,0,16,13315067198,24597,MD,21162
47434359,iosapp,2019-03-13T08:11:38.000Z,Delivery,credit,0,25,15557129447,64392,NY,12944
47437194,partner_website,2019-03-13T09:07:31.000Z,Delivery,cash,0,43,25689365012,33699,NJ,8007
47442942,partner_website,2019-03-13T11:01:04.000Z,Delivery,credit,0,28,25421112251,19002,WI,53235
47443107,partner_website,2019-03-13T11:04:56.000Z,Delivery,credit,0,23,18529185191,34911,MA,2122
...,...,...,...,...,...,...,...,...,...,...
47571438,iosapp,2019-03-14T20:33:11.000Z,Delivery,credit,0,19,18312751742,57072,PA,19382
47574693,partner_website,2019-03-15T07:32:12.000Z,Delivery,cash,0,17,6103411793,15639,CT,6484
47575101,androidapp,2019-03-15T07:51:01.000Z,Delivery,credit,0,37,27267518882,30054,NJ,7921


# ***Data Cleaning***

In [6]:
# Remove zero dollar txns
test=test[test['restaurant_total']>0]

In [7]:
# Variable Conversion
test['date_purchased']=pd.to_datetime(test['date_purchased'])
test['dateonly_purchased']=test['date_purchased'].dt.date
test['houronly_purchased']=test['date_purchased'].dt.hour
test['datehour_purchased']=test['dateonly_purchased'].astype(str) + "-" + test['houronly_purchased'].astype(str)

test['restaurant_total']=test['restaurant_total'].astype(float)

In [8]:
# Removing Duplicate Txns - i.e. those made within the same hour at the same shop by a given customer. Keeping last txn only.
test=test.drop_duplicates(subset=['customer', 'shops_id', 'datehour_purchased'], keep='last')


# ***Data Prep - Indicators***

In [9]:
 #Enumerating Purchases
 test.sort_values(by=['customer','date_purchased'])
 
test = test.assign(order_enum=test.groupby('customer')['datehour_purchased'].cumcount()+1)
test['total_order']=test.groupby('customer')['order_id'].transform('count')

In [22]:
test['bin_orders']=np.where(test['total_order']<7,test['order_enum'],7)

In [23]:
# Calculating Journey Data
#df.groupby(['site', 'country'])['score'].diff().fillna(0)
test['time_between_purch']=test.groupby(['customer'])['date_purchased'].diff()
test['spend_growth']=test.groupby(['customer'])['restaurant_total'].diff()


In [20]:
test[test['customer']==6196987925]

,order_id,source,date_purchased,shipping_type,payment_method,promo_value,restaurant_total,customer,shops_id,state,postal_code,dateonly_purchased,houronly_purchased,datehour_purchased,order_enum,total_order,bin_orders,time_between_purch,spend_growth
25,47474511,slicelife,2019-03-13 16:33:51+00:00,Delivery,cash,0,21,6196987925,27246,WA,98101,2019-03-13,16,2019-03-13-16,1,3,1,NaT,nan
48,47513940,slicelife,2019-03-14 13:15:11+00:00,Delivery,cash,0,21,6196987925,27246,WA,98101,2019-03-14,13,2019-03-14-13,2,3,2,0 days 20:41:20,0
90,47566653,slicelife,2019-03-14 18:44:08+00:00,Delivery,cash,0,19,6196987925,27246,WA,98101,2019-03-14,18,2019-03-14-18,3,3,3,0 days 05:28:57,-2


# ***Preparing Shop-Level Variables***